In [ ]:
#to get the files from google drive folder
!pip install kora -q
from kora import drive
drive.link_nbs()

     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
Mounted at /content/drive


In [ ]:
import pandas as pd
import requests
import io
import numpy as np  
from datetime import date, timedelta
import re
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#from github import Github
#import github
import torch
import torch.nn as nn
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader
# Import nn.functional
import torch.nn.functional as F
import torch.optim as optim
from typing import Union, Tuple
import os
import sys
import time
from collections import OrderedDict
from sklearn.preprocessing import MinMaxScaler
from statistics import mean
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import math
import random
import imageio
from data_preparation import create_dataset, data_preparation
from model_fit_code import fit
#from sklearn.metrics import mean_absolute_percentage_error
matplotlib.style.use('seaborn')
%matplotlib inline

In [ ]:
# To flatten the out put of convolution block
class Flatten(nn.Module):
    def forward(self, x):
        x = x.view(x.size()[0], -1)
        return x
# we have gernlised the CNN model here
class CNNModel(nn.Module):
  def __init__(self, time_step,n_layers,vector_length,kernel_size):
    super(CNNModel, self).__init__()
    self.time_step=time_step
    self.n_layers=n_layers
    in_channels=1
    out_channels=16
    layers=[]
    dimension=vector_length
    for l in range(self.n_layers): #to iterate for number of layers
        cnn_1d_layer=nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding="same") #1 d convolution
        #dimension=dimension-kernel_size+2*1+1
        #if dimension>1:
        layers.append(cnn_1d_layer)
        if l%2==0:
          layers.append(nn.Tanh()) #tanh activation function
        else:
          layers.append(nn.ELU(inplace=True)) #Elu activation function
        in_channels=out_channels
        out_channels=out_channels*2
    layers.append(Flatten()) #flatten layer
    #layers.append(nn.Dropout(p=0.2)) #dropout layer 
    self.body = nn.Sequential(*layers) 
    #print('dm',dimension,out_channels)
    out=int(vector_length*(out_channels/2))
    self.head=nn.Linear(out, 1) #fully connected layer 
  def forward(self, x):
    b, features, look_back = x.shape
    #print(b,n_steps, features)
    x = x.reshape([b,1,features*look_back])
    y = self.body(x)
    #print(y.shape)
    #print('re',y.view(len(y),-1).shape)
    return self.head(y.view(len(y),-1))

In [ ]:
Shortlisted_States=['Karnataka','Maharashtra','Uttar-Pradesh','Kerala','Tamil-Nadu']# highly impacted states relatively
results_cnn=[]
cnn_models=[]
for state in Shortlisted_States: # iteration for each state
  best_models=[]
  #getting data from our github repository
  df=pd.read_csv("https://raw.githubusercontent.com/sureshkuc/Data-Science-in-Life-Science-Project/main/Indian-States-Covid19-Datasets/"+state+".csv", parse_dates=["Date"]).drop(columns =["Unnamed: 0"])
  df = df[df["Date"] > "2020-03-10"] # selecting data from march 10, 2020 onwards
  df = df.set_index("Date")
  df = df[['Confirmed', 'Recovered', 'Deceased', 'New_Confirmerd', 'New_Deaths', 'New_Recovered']] #selected features
  #print(df.describe())

  time_step=[5,7,15,30] # time step list
  Number_of_feature=[1,2,3,4,5,6] # number of feature list
  multi_feature=True
  min_error=np.iinfo(0).max  # min error initialization
  cnn_best_model={}
  temp_result=[]
  for n_f in Number_of_feature: # feature iteration
    for t_s in time_step: #time step iteration
      #data preprocessing
      train_loader, test_loader,scaler = data_preparation(df, scaling_range=(0,1),time_step=t_s,number_feature=n_f, response_variable_index=0,data_split_ratio=0.8)
      for n_layers in range(2,5,1): # number of layers iteration
        for kernel_size in range(1,5,1): #kernel size iteration
          
          max_epochs=10 
          # below seed is used to reproduce the results
          #random.seed(42)
          #torch.manual_seed(42)
          #np.random.seed(42)
          #CNN model with L1 loss
          #best_model=Call_CNN_model(state,dataset=(train_loader, test_loader), lr=1e-2,criterion=nn.L1Loss(),max_epochs=max_epochs)
          CNN_model =  CNNModel(t_s,n_layers,t_s*n_f,kernel_size)
          cuda=torch.cuda.is_available()
          if cuda:
            CNN_model = CNN_model.cuda()
            print("cuda",cuda)
          optimizer = optim.SGD(CNN_model.parameters(), lr=1e-2, momentum=0.9)
          train_losses,test_losses, best_model = fit(CNN_model, optimizer, nn.L1Loss(),(train_loader, test_loader), max_epochs=max_epochs, cuda=cuda)
          end = time.time()
          #print(f'\nTraining took {end-start}s!')
          #plot_loss(max_epochs,train_losses,test_losses,model_name='CNN for '+state)
          CNN_model =  CNNModel(t_s,n_layers,t_s*n_f,kernel_size)
          CNN_model.load_state_dict(best_model)
          CNN_model.eval()
          test_x,test_y=test_loader
          predictions=CNN_model(test_x)
          test_y=test_y.cpu().detach().numpy()
          predictions=predictions.cpu().detach().numpy()
          #predictions = scaler.inverse_transform(predictions)
          #target = scaler.inverse_transform(target)
          mae=mean_absolute_error(test_y,predictions)
          rmse=math.sqrt(mean_squared_error(test_y,predictions))
          #rmse=math.sqrt(mean_squared_error(test_y,predictions))
          r2s=r2_score(test_y,predictions)
          if rmse<min_error:
            min_error=rmse
            cnn_best_model=best_model
            temp_result=[state,n_f,t_s,n_layers,kernel_size,mae,rmse,r2s]
            
          #mape=mean_absolute_percentage_error(test_y,predictions)
          
          
          print(state,'n_f',n_f,'t_s',t_s,'n_layers',n_layers,'kernel_size',kernel_size,mae,rmse,r2s)
          results_cnn.append([state,n_f,t_s,n_layers,kernel_size,mae,rmse,r2s])
      #CNN_model =  CNNModel(t_s,n_layers,t_s*n_f,kernel_size)
      #CNN_model.load_state_dict(best_model)
  
  cnn_models.append(cnn_best_model) 

(384, 5, 1)
(384, 5, 1) (384, 1)
cuda True
Epoch: 0/10  Loss: 0.112294 Test loss: 0.010349Karnataka n_f 1 t_s 5 n_layers 2 kernel_size 1 0.109343626 0.13226652119170124 0.6860489485010031
cuda True
Epoch: 0/10  Loss: 0.111606 Test loss: 0.001940Karnataka n_f 1 t_s 5 n_layers 2 kernel_size 2 0.21619691 0.24127418831643369 -0.044681231231511465
cuda True
Epoch: 0/10  Loss: 0.093538 Test loss: 0.026078Karnataka n_f 1 t_s 5 n_layers 2 kernel_size 3 0.063344374 0.07311294596919697 0.9040708428266812
cuda True
Epoch: 0/10  Loss: 0.130036 Test loss: 0.002252Karnataka n_f 1 t_s 5 n_layers 2 kernel_size 4 0.15406409 0.17353245953647212 0.4595898611003242
cuda True
Epoch: 0/10  Loss: 0.135640 Test loss: 0.071011Karnataka n_f 1 t_s 5 n_layers 3 kernel_size 1 0.110233285 0.1246268319963314 0.7212690668112551
cuda True
Epoch: 0/10  Loss: 0.113668 Test loss: 0.031389Karnataka n_f 1 t_s 5 n_layers 3 kernel_size 2 0.14122227 0.1603443506549654 0.5386087104219114
cuda True
Epoch: 0/10  Loss: 0.096041 T

In [ ]:
#storing result into data frame
df_cnn = pd.DataFrame (results_cnn,columns=['State','Number_feature','Time_Step','number_layers','kernel_size','MAE','RMSE','R2_Score'])
df_cnn.head()

In [ ]:
print(torch.__version__)

1.9.0+cu102
